# Sistema de recomendação de produtos da Amazon (Automotive_5.json) utilizando kNN para Filtro baseado em Conteúdo

In [73]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn as sns
import numpy as np
import sklearn as sklearn
import re
import math

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from pandas import DataFrame

from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn import neighbors
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

pd.set_option('float_format', '{:f}'.format)
reviews = pd.read_json('Automotive_5.json', lines = True)

#### Descrição do Dataset:
- reviewerID - ID of the reviewer, e.g. A2SUAM1J3GNN3B
- asin - ID of the product, e.g. 0000013714
- reviewerName - name of the reviewer
- vote - helpful votes of the review
- style - a dictionary of the product metadata, e.g., "Format" is "Hardcover"
- reviewText - text of the review
- overall - rating of the product
- summary - summary of the review
- unixReviewTime - time of the review (unix time)
- reviewTime - time of the review (raw)
- image - images that users post after they have received the product

In [74]:
display(reviews.head(10))

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,4,False,"05 1, 2015",A8WEXFRWX1ZHH,0209688726,{'Color:': ' AC'},Goldengate,"After I wrote the below review, the manufactur...",Works well if you place phone in horizontally ...,1430438400,NaN,NaN
1,1,True,"04 19, 2018",ABCA1A8E4DGV1,0209688726,{'Color:': ' Blue'},noe,It sucks barely picks up anything definitely n...,sucks,1524096000,NaN,NaN
2,1,True,"04 16, 2018",A1NX8HM89FRQ32,0209688726,{'Color:': ' Black'},Eduard,"Well to write a short one, it blew 2 fuses of ...",Defective,1523836800,NaN,NaN
3,3,True,"04 13, 2018",A1X77G023NY0KY,0209688726,{'Color:': ' CA'},Lauren,I have absolutely no memory of buying this but...,Looks cool! Probably works,1523577600,NaN,NaN
4,5,True,"04 8, 2018",A3GK37JO2MGW6Q,0209688726,{'Color:': ' Black'},danny,it ok it does it job,Five Stars,1523145600,NaN,NaN
5,5,True,"03 24, 2018",AIY18YON1TWJJ,0209688726,{'Color:': ' Black'},Karen H.,Have 3 big dogs. this have been great for my F...,this have been great for my Ford transit connect,1521849600,NaN,NaN
6,3,True,"03 4, 2018",A2MPTQ85HBBNG2,0209688726,{'Color:': ' Black'},Giv,"Pros: Good attachments, nice long cord, can re...",Decent car vaccuum.,1520121600,NaN,NaN
7,2,True,"03 1, 2018",A1SPIM9Y6HUUSH,0209688726,{'Color:': ' Black'},Frank W.Brodeur,I have a 2017 outback and everytime I try to u...,Two Stars,1519862400,NaN,NaN
8,4,True,"02 22, 2018",A1Q6FHU6DA643L,0209688726,{'Color:': ' Black'},nutter1,very good suction will see how it lasts,Four Stars,1519257600,NaN,NaN
9,5,True,"01 29, 2018",A3MA15RJJ59OKG,0209688726,{'Color:': ' Black'},Daryl S.,"love it,works great ! wow !!",Five Stars,1517184000,NaN,NaN


## Análise e Limpeza dos Dados

### Informações Gerais

In [75]:
reviews.shape
print("Número de Linhas: ", reviews.shape[0])
print("Número de Colunas: ", reviews.shape[1])

Número de Linhas:  1711519
Número de Colunas:  12


In [76]:
print('Colunas com valores null: \n',reviews.isnull().sum())

Colunas com valores null: 
 overall                 0
verified                0
reviewTime              0
reviewerID              0
asin                    0
style             1118104
reviewerName          140
reviewText            866
summary               342
unixReviewTime          0
vote              1520651
image             1668825
dtype: int64


In [77]:
reviews.describe()

,overall,unixReviewTime
count,1711519.000000,1711519.000000
mean,4.459656,1454262583.608596
std,1.055129,49138282.476421
min,1.000000,968889600.000000
25%,4.000000,1424908800.000000
50%,5.000000,1460678400.000000
75%,5.000000,1490659200.000000
max,5.000000,1538524800.000000


In [78]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1711519 entries, 0 to 1711518
Data columns (total 12 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   overall         int64 
 1   verified        bool  
 2   reviewTime      object
 3   reviewerID      object
 4   asin            object
 5   style           object
 6   reviewerName    object
 7   reviewText      object
 8   summary         object
 9   unixReviewTime  int64 
 10  vote            object
 11  image           object
dtypes: bool(1), int64(2), object(9)
memory usage: 145.3+ MB


In [79]:
print('Quantidade de Usuários: ', reviews['reviewerID'].nunique())
print('Quantidade de Produtos: ', reviews['asin'].nunique())

Quantidade de Usuários:  193651
Quantidade de Produtos:  79437


#### Atributos que não serão utilizados

In [80]:
reviews.drop(['reviewerName', 'reviewTime', 'unixReviewTime', 'style', 'verified', 'image' ], axis=1, inplace=True)

In [182]:
count = reviews.groupby("asin", as_index=False).count()
mean = reviews.groupby("asin", as_index=False).mean()

dfMerged = pd.merge(reviews, count, how='right', on=['asin'])
dfMerged = pd.merge(dfMerged, mean, how='right', on=['asin'])
dfMerged

,overall_x,reviewerID_x,asin,reviewText_x,summary_x,vote_x,overall_y,reviewerID_y,reviewText_y,summary_y,vote_y,overall
0,4,A8WEXFRWX1ZHH,0209688726,"After I wrote the below review, the manufactur...",Works well if you place phone in horizontally ...,NaN,71,71,71,71,11,4.042254
1,1,ABCA1A8E4DGV1,0209688726,It sucks barely picks up anything definitely n...,sucks,NaN,71,71,71,71,11,4.042254
2,1,A1NX8HM89FRQ32,0209688726,"Well to write a short one, it blew 2 fuses of ...",Defective,NaN,71,71,71,71,11,4.042254
3,3,A1X77G023NY0KY,0209688726,I have absolutely no memory of buying this but...,Looks cool! Probably works,NaN,71,71,71,71,11,4.042254
4,5,A3GK37JO2MGW6Q,0209688726,it ok it does it job,Five Stars,NaN,71,71,71,71,11,4.042254
...,...,...,...,...,...,...,...,...,...,...,...,...
1711514,5,A73IGP6LB5WDM,B01HJI17Y8,"A breeze to install, and brighter that OEM. Do...",An improvement.,NaN,8,8,8,8,0,4.500000
1711515,1,A261FYJ5MTGSUJ,B01HJI17Y8,"They're not that bright, my regular halogen ar...",One Star,NaN,8,8,8,8,0,4.500000
1711516,5,A1PSKAPFER6BZW,B01HJI17Y8,Wow! So bright. I can see so much farther. Lig...,Drastically improves night vision.,NaN,8,8,8,8,0,4.500000
1711517,5,A1MJUNTX7CTR5U,B01HJI17Y8,Way better than stock. Nice bright and white. ...,"NICE, BRIGHT AND WHITE",NaN,8,8,8,8,0,4.500000


### KNN - Utilizando palavras para recomendação

Os textos primeiro tem que ser tratados para depois conseguir utilizar o KNN. É importante notar que foram utilizados apenas produtos com mais de 50 reviews. Caso contrário, a matriz seria esparsa demais e isso iria afetar significativamente o desempenho do sistema de recomendação.

In [183]:
dfNew = dfMerged[['asin','summary_x','overall',"overall_y"]]
dfNew = dfNew.rename(columns={'overall_y': 'reviewsCount', 'summary_x': 'summary', 'overall': 'avg_rating'})
# pegando apenas os produtos com mais de 50 reviews
dfNew = dfNew[dfNew.reviewsCount >= 50]

dfNew

,asin,summary,avg_rating,reviewsCount
0,0209688726,Works well if you place phone in horizontally ...,4.042254,71
1,0209688726,sucks,4.042254,71
2,0209688726,Defective,4.042254,71
3,0209688726,Looks cool! Probably works,4.042254,71
4,0209688726,Five Stars,4.042254,71
...,...,...,...,...
1710603,B01HCIGK4W,One of the lights was defective. Manufacturer...,4.440000,50
1710604,B01HCIGK4W,Super Nova bright...and great quality,4.440000,50
1710605,B01HCIGK4W,I was satisfied but not anymore after a year,4.440000,50
1710606,B01HCIGK4W,Iffy Product....Loose in Housing/Socket....Bew...,4.440000,50


In [184]:
dfProductReview = reviews.groupby("asin", as_index=False).mean()
ProductReviewSummary = dfNew.groupby("asin")["summary"].apply(list)
ProductReviewSummary = pd.DataFrame(ProductReviewSummary)

ProductReviewSummary

,summary
asin,
0209688726,[Works well if you place phone in horizontally...
B0000223J0,"[nice, thick, buffing pad . . ., nice product ..."
B00002243Z,"[Five Stars, Five Stars, Best cables I've ever..."
B00002N57X,"[Five Stars, Direct replacement working good s..."
B00004RB1T,[Air Filter for Briggs & Stratton 19.5 Twin Pi...
...,...
B01HAWMBH0,"[Correct fit, good value, Easy, effective, gre..."
B01HAXAYSM,"[worth money, Good mirrors, Five Stars, Rear v..."
B01HB1OYUC,"[Looks Durable, Solid piece of equipment, Manl..."


In [185]:
dfProductReview = pd.merge(ProductReviewSummary, dfProductReview, on="asin", how='inner')

dfProductReview 

,asin,summary,overall
0,0209688726,[Works well if you place phone in horizontally...,4.042254
1,B0000223J0,"[nice, thick, buffing pad . . ., nice product ...",4.741379
2,B00002243Z,"[Five Stars, Five Stars, Best cables I've ever...",4.564706
3,B00002N57X,"[Five Stars, Direct replacement working good s...",4.537500
4,B00004RB1T,[Air Filter for Briggs & Stratton 19.5 Twin Pi...,4.842105
...,...,...,...
6366,B01HAWMBH0,"[Correct fit, good value, Easy, effective, gre...",4.810127
6367,B01HAXAYSM,"[worth money, Good mirrors, Five Stars, Rear v...",4.254902
6368,B01HB1OYUC,"[Looks Durable, Solid piece of equipment, Manl...",4.653465
6369,B01HBFKNFS,"[If you're meticulous DON'T buy these!, Five S...",4.671053


#### Limpando texto e fazendo o stemming

In [190]:
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()
# Usando NLTK's PorterStemmer

def stemming_tokenizer(str_input):
    reviewsText = ''
    for review in str_input:
        if not type(review) == str:
            continue
        words = re.sub(r"[^A-Za-z0-9\-]", " ", review).lower().split()
        for word in words:
            reviewsText += porter_stemmer.stem(word) + ' '
    return reviewsText[:len(reviewsText)-1]

In [191]:
dfProductReview["summaryAll"] = dfProductReview["summary"].apply(stemming_tokenizer)
dfProductReview = dfProductReview.drop_duplicates(['overall'], keep='last')
dfProductReview = dfProductReview.reset_index()

dfProductReview

,index,asin,summary,overall,summaryAll
0,0,0209688726,[Works well if you place phone in horizontally...,4.042254,work well if you place phone in horizont lands...
1,2,B00002243Z,"[Five Stars, Five Stars, Best cables I've ever...",4.564706,five star five star best cabl i ve ever use ga...
2,6,B00004Y774,"[Top quality as always, Works pretty good, For...",4.344538,top qualiti as alway work pretti good for smal...
3,9,B000056OQL,"[Five Stars, One Star, Four Stars, minus 1 sta...",3.692308,five star one star four star minu 1 star but t...
4,10,B00008RW9U,"[Works for light dust, but is really only the ...",4.483660,work for light dust but is realli onli the fir...
...,...,...,...,...,...
3606,6366,B01HAWMBH0,"[Correct fit, good value, Easy, effective, gre...",4.810127,correct fit good valu easi effect great valu p...
3607,6367,B01HAXAYSM,"[worth money, Good mirrors, Five Stars, Rear v...",4.254902,worth money good mirror five star rear view mi...
3608,6368,B01HB1OYUC,"[Looks Durable, Solid piece of equipment, Manl...",4.653465,look durabl solid piec of equip manli kit buil...
3609,6369,B01HBFKNFS,"[If you're meticulous DON'T buy these!, Five S...",4.671053,if you re meticul don t buy these five star wo...


#### Utilizando CountVectorizer

In [202]:
allReviews = dfProductReview["summaryAll"] 
countVector = CountVectorizer(max_features = 500, stop_words='english') # utilizando countVector do sklearn para transformar as palavras em vetores, selecionando apenas as 500 palavras mais frequentes
transformedReviews = countVector.fit_transform(allReviews) 
dfReviews = DataFrame(transformedReviews.A, columns=countVector.get_feature_names_out())
dfReviews = dfReviews.astype(int)

dfReviews.head(15)

,10,12,150,1500,20,2001,2002,2003,2004,2005,...,wonder,work,worth,wow,wrangler,wrench,wrong,ye,year,yellow
0,0,0,0,0,0,0,0,0,0,0,...,0,7,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,2,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,7,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,3,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,32,0,0,0,0,0,2,4,0
5,0,0,0,0,0,0,0,0,0,0,...,0,16,0,0,0,0,0,0,1,0
6,0,0,0,0,0,0,0,1,0,0,...,1,5,0,0,0,0,0,1,5,0
7,1,0,0,0,0,0,0,0,0,0,...,0,40,1,1,0,0,0,0,2,0
8,0,0,0,0,0,0,0,0,0,0,...,0,19,2,0,0,0,0,0,0,0
9,3,2,0,0,0,0,0,0,0,0,...,1,30,0,0,0,0,0,0,6,0


#### Utilizando TF-IDF

In [203]:
allReviews = dfProductReview["summaryAll"] 
tfidf = TfidfVectorizer(max_features = 500, stop_words='english') # utilizando tfidf do sklearn para transformar as palavras em vetores, selecionando apenas as 500 palavras mais frequentes
transformedReviews = tfidf.fit_transform(allReviews) 
dfReviews = DataFrame(transformedReviews.A, columns=tfidf.get_feature_names_out())
dfReviews = dfReviews.astype(float)

dfReviews.head(15)

,10,12,150,1500,20,2001,2002,2003,2004,2005,...,wonder,work,worth,wow,wrangler,wrench,wrong,ye,year,yellow
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.174118,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.032203,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.018784,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019554,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.087497,0.021221,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.055284,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.017751,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.201843,0.000000,0.000000,0.000000,0.000000,0.000000,0.040303,0.052530,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.159706,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020782,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016907,0.000000,0.000000,...,0.015409,0.026505,0.000000,0.000000,0.000000,0.000000,0.000000,0.016936,0.055185,0.000000
7,0.014828,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.210762,0.008945,0.014995,0.000000,0.000000,0.000000,0.000000,0.021941,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.111607,0.019945,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.038748,0.031474,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.013341,0.137690,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.057335,0.000000


#### Criando train test

In [204]:
train_data, test_data = train_test_split(dfReviews, test_size=0.25)
train_data = train_data.to_numpy()
test_data = test_data.to_numpy()


#### Aplicação do Modelo

In [205]:
neighbor = NearestNeighbors(n_neighbors=3, algorithm='ball_tree').fit(train_data)
distances, indices = neighbor.kneighbors(train_data)

#### Exemplo de produto relacionado

In [206]:
vizinhos = neighbor.kneighbors([train_data[0]])
relacionados = vizinhos[1]
prim = relacionados[0][0]

print("Produto =", dfProductReview["asin"][train_data.shape[0] + 0], "- Produto mais relacionado:", dfProductReview["asin"][prim])


Produto = B00MU2NY6S - Produto mais relacionado: 0209688726


#### Predição do rating do produto

In [207]:
train_target, test_target = train_test_split(dfProductReview['overall'], test_size=0.25)
# colocando os valores de train e test como inteiros pois o rating é inteiro
train_target = train_target.astype(int)
test_target = test_target.astype(int)

knn = neighbors.KNeighborsClassifier(3, weights='distance')
knn.fit(train_data, train_target)
knnpredict = knn.predict(test_data)

print(classification_report(test_target, knnpredict))

              precision    recall  f1-score   support

           2       0.00      0.00      0.00         1
           3       0.23      0.10      0.14       105
           4       0.89      0.96      0.92       797

    accuracy                           0.86       903
   macro avg       0.37      0.35      0.35       903
weighted avg       0.81      0.86      0.83       903



#### Precisão do modelo

In [208]:
accuracy_score(test_target, knnpredict)

0.8560354374307863

#### RMSE do modelo

In [209]:
math.sqrt(mean_squared_error(test_target, knnpredict))

0.38808385800190015